# GADTs Derivation Using Yoneda Lemma
##### (http://www.haskellforall.com/2012/06/gadts.html)
GADTs (Generalized Algebraic Data Types) allow for value constructors of a data type to be defined as a type signature. This allows us to add type annotations to a value, for example the safe-head list:

In [46]:
:ext GADTs

data Z
data S n

data SafeList a n where
  Nil :: SafeList a Z
  Cons :: a -> SafeList a n -> SafeList a (S n)

safeHead :: SafeList a n -> a
safeHead (Cons a _) = a

In the abscence of the `GADTs` language extension we only need `RankNTypes` to achieve this behaviour. We do this through application of the Yoneda Lemma.

In [49]:
-- Hom-functor aka Reader Monad
type Hom a = (->) a

-- Natural transformation between functors f and g
type Nat f g = forall x. f x -> g x

to :: Functor f => Nat (Hom a) f -> f a
to nat = nat id

from :: Functor f => f a -> Nat (Hom a) f
from fa atox = fmap atox fa

-- to . from = id
-- from . to = id

Yoneda tells us that there exists a natural isomorphism between the natural transformation involving the hom-functor (`forall f b. (a -> b) -> f b`) and the container `f a` where `f` is an arbitrary functor. 

We can use this to transform the GADTs constructors into isomorphic regular constructors. Starting with `Nil :: SafeList a Z`, presuming that `SafeList a` is a functor, then `(Z -> n) -> SafeList a n ~ SafeList a Z`.

Similiarly, `Cons` can be transformed into `a -> SafeList a m -> (S m -> n) -> SafeList a n`.

This gives us constructors that have polymorphic type variables in their result type, so we can then write it as a regular data type:

In [30]:
data SafeList' a n
  = Nil' (Z -> n)
  | forall m. Cons' a (SafeList' a m) (S m -> n)

instance Functor (SafeList' a) where
  fmap f (Nil' k) = Nil' (f . k)
  fmap f (Cons' x xs k) = Cons' x xs (f . k)

This converts the `Nat` 'phantom type' into a concrete value-level dependency.

We can convert back and forth to the GADT representation:

In [48]:
:t to Nil'

:t to (Cons' 'A' (to Nil'))

:t (from (to Nil'))

:t from Nil


to Nil' :: forall a. SafeList' a Z

to (Cons' 'A' (to Nil')) :: SafeList' Char (S Z)

(from (to Nil')) :: forall x a. Hom Z x -> SafeList' a x

from Nil :: forall a x. Functor (SafeList a) => Hom Z x -> SafeList a x

Now just define some smart constructors, analogous to what we get with the GADTs:

In [45]:
nil = to Nil'

cons x xs = to (Cons' x xs)

:t cons 'A' nil

cons 'A' nil :: SafeList' Char (S Z)

`SafeList' a` is not useful as a functor because it is mapping over the length variable, not the value variable. We can derive the useful functor instances by flipping the type variables but we lose the ability to apply the Yoneda Lemma.

In [53]:
data List n a where
  Nil :: List Z a
  Cons :: a -> List n a -> List (S n) a
  
instance Functor (List n) where
  fmap f Nil = Nil
  fmap f (Cons a as) = Cons (f a) (fmap f as)
  
:t from Nil
:t from (Cons 'A' Nil)

from Nil :: forall a x. Hom a x -> List Z x

from (Cons 'A' Nil) :: forall x. Hom Char x -> List (S Z) x

In [64]:
data List' n a
  = Nil' (Z -> n)
  | forall m. Cons (S m -> n) a (List' m a)

instance Functor (List' n) where
  fmap f (Nil' k) = Nil' k
  fmap f (Cons k a as) = Cons k (f a) (fmap f as)

In [62]:
nil = Nil' id
cons a as = Cons id a as

:t cons 'A' nil

cons 'A' nil :: List' (S Z) Char